#### Watch the Getting Started Quick Start [Video](https://aka.ms/SK-Getting-Started-Notebook)

> [!IMPORTANT]
> You will need an [.Net 7 SDK](https://dotnet.microsoft.com/en-us/download) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .Net Interactive

**Step 1**: Configure your AI service credentials

Use [this notebook](0-AI-settings.ipynb) first, to choose whether to run these notebooks with OpenAI or Azure OpenAI,
and to save your credentials in the configuration file.

In [ ]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs 

**Step 2**: Import Semantic Kernel SDK from NuGet

In [ ]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 0.17.230626.1-preview"

**Step 3**: Instantiate the Kernel

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Memory;

//Create Kernel builder
var builder = new KernelBuilder();

In [ ]:
// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
{
	builder.WithAzureTextEmbeddingGenerationService("text-embedding-ada-002", azureEndpoint, apiKey);
	if(model.Contains("gpt"))
	{
		builder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);
	}
	else
	{
		builder.WithAzureTextCompletionService(model, azureEndpoint, apiKey);
	}
}
else
{
	builder.WithOpenAITextEmbeddingGenerationService("text-embedding-ada-002", apiKey);
	builder.WithOpenAITextCompletionService(model, apiKey, orgId);
}

builder.WithMemoryStorage(new VolatileMemoryStore());

var kernel = builder.Build();


**Step 4**: Load sample code as context

In [ ]:
using System.IO;
using System;
using System.Threading.Tasks;
using Microsoft.SemanticKernel.Text;

const string MemoryCollectionName = "SampleCode";
const int MaxFileSize = 2048;
const int MaxTokens = 1024;

async Task SummarizeCodeFileAsync(string filePath)
{
	string code = File.ReadAllText(filePath);

	if (code != null && code.Length > 0)
	{
		var fileInfo = new FileInfo(filePath);
		var fileName = fileInfo.Name;
		if (code.Length > MaxFileSize)
		{
			var extension = fileInfo.Extension;
			List<string> lines;
			List<string> paragraphs;

			switch (extension)
			{
				case ".md":
					{
						lines = TextChunker.SplitMarkDownLines(code, MaxTokens);
						paragraphs = TextChunker.SplitMarkdownParagraphs(lines, MaxTokens);

						break;
					}
				default:
					{
						lines = TextChunker.SplitPlainTextLines(code, MaxTokens);
						paragraphs = TextChunker.SplitPlainTextParagraphs(lines, MaxTokens);

						break;
					}
			}

			for (int i = 0; i < paragraphs.Count; i++)
			{
				await kernel.Memory.SaveInformationAsync(
					MemoryCollectionName,
					text: $"{code}",
					id: fileName);
			}
		}
		else
		{
			await kernel.Memory.SaveInformationAsync(
				MemoryCollectionName,
				text: $"{code}",
				id: fileName);
		}
	}
}

async Task SummarizeCodeDirectoryAsync(string directoryPath, string searchPattern)
{
	string[] filePaths = Directory.GetFiles(directoryPath, searchPattern, SearchOption.AllDirectories);

	if (filePaths != null && filePaths.Length > 0)
	{
		Console.WriteLine("Found {0} files to summarize", filePaths.Length);

		foreach (string filePath in filePaths)
		{

			await SummarizeCodeFileAsync(filePath);
		}
	}
}

await SummarizeCodeDirectoryAsync("../sample/Comet.Sample/Views", "*.cs");

**Step 5**: Answer the question based on the context.


In [ ]:
async Task AnswerTheQuestion(string question)
{
    string skPrompt = @"
Read this demo code
===========
 {{$demo}}. 
===========
Then answer this question with code only, all the code should use from the demo.
{{$input}}
";

    var codeFunc = kernel.CreateSemanticFunction(skPrompt, 
                                                maxTokens: 200, 
                                                temperature: 0, 
                                                topP: 0.5);
    var response = kernel.Memory.SearchAsync(MemoryCollectionName, question);
    await foreach (var item in response)
    {
        var context = kernel.CreateNewContext();
        context["demo"] = item.Metadata.Text;
        context["input"] = question;
        var answer = await codeFunc.InvokeAsync(context);
        Console.WriteLine(answer + " : " + item.Relevance);
    }
}

**Ask the question**

In [ ]:
await AnswerTheQuestion("How to create a button");